<a href="https://colab.research.google.com/github/RachelRamirez/misclassification_matrix/blob/main/Potential_Misclassification_Cost_Matrix_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What happens when I replace Weighted_Matrix with w_array[7, 9] = 1

Now that I've shown the TB and ISantaro methods are equivalent when seeded properly, I believe what I need to do is train a "good" neural network with the same seed 30 times to get an idea of what the typical confusion matrix looks like.  I will run the code within Colab 30 times without restarting because from previous reproducibility studies if I restart and run-all I will get the same results.  This time, I'm interested in the typical variety you can get on a confusion matrix when the neural network is seeded the same way each time, that way when I gve it another cost-matrix to train on, and run that 30 times I can do a more informative comparison of the results.  Since the Isantaro and TB methods were identical I went with the Isantaro method because it was simpler, more efficient, and seemed less time consuming. 

Changes from previous Reproducibility notebook:
1. Dropout is back in.
2. Batch Size is not as large to help with variety.
3. Num of Epochs is more than 4 now that I care about achieving good overall accuracy
4. Callback for EarlyStop added
5. Model Shuffle during Fit is still False (I'm calling it out to see if I need to change that)
6. but Model.Fit(use multiprocessors = True)
 

Remember to change the [Admin File stuff]  below and the Weight Matrix before Running

### Reproducible Seeds

In [1]:
#For Reproducibility
import numpy as np
# np.random.seed(1337)  # for reproducibility

import tensorflow as tf
# tf.random.set_seed(33)

import random as python_random
# python_random.seed(4)

# https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed
tf.keras.utils.set_random_seed(342) #Possibly use next iteration if the above doesn't work


# Running more than once causes variation.  try adding this:
# Set seed value
seed_value = 56
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

print("TF version: " , tf.__version__ )
print("Keras version: " , tf.keras.__version__ )

TF version:  2.9.2
Keras version:  2.9.0


### Import rest of Library

In [2]:

# from https://github.com/keras-team/keras/issues/2115#issuecomment-204060456
# witha correction on the weighted function in the middle 

'''Train a simple deep NN on the MNIST dataset.
Get to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function  #do i still need this?
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import keras.backend as K
from itertools import product
import functools
from functools import partial
from time import ctime
from time import sleep
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
from sklearn.metrics import confusion_matrix

## MORE REPEATABILITY STUFF NEEDED - If theres a way to update this to V2 of Tensorflow great, otherwise I had to use TF 1.0 code
# 5. Configure a new global `tensorflow` session (https://stackoverflow.com/questions/50659482/why-cant-i-get-reproducible-results-in-keras-even-though-i-set-the-random-seeds)
# from keras import backend as K


#I believe thecode below is to help things be repeatable each time different sections in my google colab notebook execute
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

### Define batch, epochs, and format data

In [3]:
batch_size = 256 # I originally had it very  high batch size to reduce the variation in the data each batch and hope it makes the model training more nearly identical which it did, then i bring it back down to something reasonable to get better results training the NN
nb_classes = 10
nb_epoch = 15

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()


X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


11490434/11490434 [==============================] - 1s 0us/step
60000 train samples
10000 test samples


### Define weighted_categorical_crossentropy()

In [4]:

# # https://github.com/keras-team/keras/issues/2115#issuecomment-207765342

# def w_categorical_crossentropy(y_true, y_pred, weights):
#     nb_cl = len(weights)
#     final_mask = K.zeros_like(y_pred[:, 0])
#     y_pred_max = K.max(y_pred, axis=1)
#     y_pred_max = K.expand_dims(y_pred_max, 1)
#     y_pred_max_mat = K.equal(y_pred, y_pred_max)
 
#     for c_t, c_p in product(range(nb_cl), range(nb_cl)):
#         final_mask += (K.cast(weights[c_t, c_p],K.floatx()) * K.cast(y_pred_max_mat[:, c_p] ,K.floatx())* K.cast(y_true[:, c_t],K.floatx()))
    
#     # result = K.categorical_crossentropy(y_true, y_pred)*final_mask
#     # tf.print(result, "Show Result of CE * Final_Mask")  #this was basically useless to display, and it showed like, 500 lines of print statements each epoch

#     return K.categorical_crossentropy(y_true, y_pred)*final_mask   #I changed the order of y_true and y_pred




### Define Same Model but use normal Categorical CrossEntropy with no extra cost-matrix of Weights

In [5]:
def normal_method():

  model = Sequential()
  model.add(Dense(512, input_shape=(784,) ,kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(Dense(512, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(Dense(10, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model.add(Activation('softmax'))

  rms = RMSprop()
  # model.compile(loss=ncce, optimizer=rms)
  model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=rms, metrics='categorical_accuracy', )

  #add early_stop to prevent overfittings
  # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  model_history = model.fit(X_train, Y_train,
            batch_size=batch_size, epochs=nb_epoch, verbose=2,
            validation_data=(X_test, Y_test),shuffle=False, use_multiprocessing=True
            , callbacks = [callback])

  
  # model.evaluate(X_test, Y_test, verbose=1)  # I know this isn't the typical use of train/val/test sets, please dont' comment on that
  
  #Predict
  y_prediction = model.predict(X_test)
  y_prediction  = np.argmax(y_prediction, axis=1)
  # Y_prediction = np_utils.to_categorical(y_prediction, nb_classes) #If I want to do SparseCategoricalAccuracy

  #Create confusion matrix and normalizes it over predicted (columns)
  # result = confusion_matrix(y_test, y_prediction , normalize='pred')  #if I want percentages instead of raw counts

  
  cm = confusion_matrix(y_test, y_prediction)
  cm = pd.DataFrame(cm, range(10),range(10))

  #This shows a pretty confusion matrix which I don't neeed to show right now
  # plt.figure(figsize = (10,10))
  # sns.heatmap(cm, annot=True, annot_kws={"size": 12}) # font size
  # plt.show()
  # cm_normal = cm

  return cm


### Weighted Categorical Cross Entropy Function

In [6]:
# from https://github.com/keras-team/keras/issues/2115#issuecomment-815825633 from Isaranto

def weighted_categorical_crossentropy_new(y_true, y_pred, weights):
          idx1 = K.argmax(y_pred, axis=1)
          idx2 = K.argmax(y_true, axis=1)
          mask = tf.gather_nd(weights, tf.stack((idx1, idx2), -1))
          return K.categorical_crossentropy(y_true, y_pred) * mask

### *Change the Weight Matrix

In [7]:
 #what does ncce stand for?

 
def isaranto_method():
  w_array = np.ones((10,10))
  w_array[7, 9] = 1     #HAVE YET TO SEE Whether this is REAL 7 PREDICTED AS 9 or vs vsa
  # w_array = w_array - np.eye(10)
  # print("W_array:  ", w_array)

  weighted_list = w_array.tolist()

  wcce = partial(weighted_categorical_crossentropy_new, weights=weighted_list)
  wcce.__name__ ='w_categorical_crossentropy'

  model3 = Sequential()
  model3.add(Dense(512, input_shape=(784,), kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('relu'))
  model3.add(Dropout(0.2))
  model3.add(Dense(512, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('relu'))
  model3.add(Dropout(0.2))
  model3.add(Dense(10,kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('softmax'))

  rms = RMSprop()

  model3.compile(loss=wcce, optimizer=rms,  metrics='categorical_accuracy',)
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  model3_history = model3.fit(X_train, Y_train,
            batch_size=batch_size, epochs=nb_epoch, verbose=2,
            validation_data=(X_test, Y_test), shuffle=False, use_multiprocessing=True
            ,callbacks = [callback]
            )

 

  #Predict
  y_prediction = model3.predict(X_test)
  y_prediction  = np.argmax(y_prediction, axis=1)
  # Y_prediction = np_utils.to_categorical(y_prediction, nb_classes)

  #Create confusion matrix and normalizes it over predicted (columns)
  # result = confusion_matrix(y_test, y_prediction , normalize='pred')

  

  cm3 = confusion_matrix(y_test, y_prediction)
  cm3 = pd.DataFrame(cm3, range(10),range(10))
  # plt.figure(figsize = (10,10))
  # cm3
  # sns.heatmap(cm2, annot=True, annot_kws={"size": 12}) # font size
  # plt.show()

  # cm_using_weighted_new = cm3

  return cm3

### *Keep Track of Experimental Admin Stuff

> Change the file extension name to match the weights



In [8]:
# Experimental Admin Stuff
from datetime import date

today = date.today()

# # dd/mm/YY
# d1 = today.strftime("%d/%m/%Y")
# print("d1 =", d1)

# # Textual month, day and year	
# d2 = today.strftime("%B %d, %Y")
# print("d2 =", d2)

# # mm/dd/y
# d3 = today.strftime("%m/%d/%y")
# print("d3 =", d3)

# # Month abbreviation, day and year	
# d4 = today.strftime("%b-%d-%Y")
# print("d4 =", d4)

file_date = today.strftime("%Y_%m_%d")
# print("d5 =", d5)

file_extension = "w[7,9]_1"



np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)

In [9]:
cm = np.zeros([10,10])
combined_cms = np.empty((1,100))

for i in range(0,30):
  print(i)
  cm2 =  isaranto_method()    #Individual CM
  print("CM: \n", cm2)
  # cm += cm2                   #Aggregating for an Average
  cm2_array = np.asarray(cm2)  #Indiv CM as array for storing
  combined_cms = np.vstack((combined_cms,cm2_array.reshape((1,100))))

# cm_new = cm/30

0
Epoch 1/15
235/235 - 21s - loss: 0.3032 - categorical_accuracy: 0.9058 - val_loss: 0.2383 - val_categorical_accuracy: 0.9259 - 21s/epoch - 89ms/step
Epoch 2/15
235/235 - 7s - loss: 0.1173 - categorical_accuracy: 0.9645 - val_loss: 0.1106 - val_categorical_accuracy: 0.9666 - 7s/epoch - 31ms/step
Epoch 3/15
235/235 - 6s - loss: 0.0788 - categorical_accuracy: 0.9756 - val_loss: 0.0989 - val_categorical_accuracy: 0.9701 - 6s/epoch - 24ms/step
Epoch 4/15
235/235 - 7s - loss: 0.0593 - categorical_accuracy: 0.9825 - val_loss: 0.1007 - val_categorical_accuracy: 0.9699 - 7s/epoch - 30ms/step
Epoch 5/15
235/235 - 6s - loss: 0.0480 - categorical_accuracy: 0.9842 - val_loss: 0.0817 - val_categorical_accuracy: 0.9772 - 6s/epoch - 24ms/step
Epoch 6/15
235/235 - 7s - loss: 0.0393 - categorical_accuracy: 0.9876 - val_loss: 0.0869 - val_categorical_accuracy: 0.9751 - 7s/epoch - 29ms/step
Epoch 7/15
235/235 - 6s - loss: 0.0325 - categorical_accuracy: 0.9897 - val_loss: 0.0983 - val_categorical_accurac

#### Save the 30 confusion matrices

In [10]:
#Save 30 confusion matrices

import pickle
file_name = "30CM_" + file_extension + "_" + file_date + "_.pkl"
print(file_name)

with open(file_name, 'wb') as file:
      
    # A new file will be created
    pickle.dump(combined_cms, file)




# Open the file in binary mode
with open(file_name, 'rb') as file:
      
    # Call load method to deserialze
    var = pickle.load(file)
  
    print(var)


from google.colab import files
files.download( file_name )  

30CM_w[7,9]_1_2023_02_09_.pkl
[[6.95e-310 6.95e-310 6.02e-316 ... 0.00e+000 7.67e+170 5.99e-316]
 [9.71e+002 1.00e+000 1.00e+000 ... 1.00e+000 1.00e+000 9.89e+002]
 [9.73e+002 1.00e+000 0.00e+000 ... 0.00e+000 0.00e+000 9.98e+002]
 ...
 [9.73e+002 0.00e+000 1.00e+000 ... 0.00e+000 1.00e+000 9.91e+002]
 [9.74e+002 1.00e+000 0.00e+000 ... 1.00e+000 0.00e+000 9.92e+002]
 [9.69e+002 1.00e+000 0.00e+000 ... 1.00e+000 0.00e+000 9.87e+002]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# How many categories are there in the test set?

truth_num_per_category = Y_test.sum(axis=0)

# Analyze 

I am now going to load the Average CM and try to get it in a format where I can make it a 1x100 and load all 30 CMs so that we can visualize their distributions in a a big histogram_matrix. At this point the Google Colab variables are gone and I have to reoad them 

In [12]:
# I need to remove the first placeholder row of zeros
combined_cms = combined_cms[1:31]

In [13]:
import numpy as np
import pandas as pd
empty_cm = np.zeros((10,10))  #why didn't this work with EMPTY instead of zeros?!!?
empty_cm=pd.DataFrame(empty_cm)

empty_cm.columns = ['0p', '1p', '2p', '3p', '4p', '5p', '6p', '7p', '8p', '9p']
empty_cm.index = ['0t', '1t', '2t', '3t', '4t', '5t', '6t', '7t', '8t', '9t']

# print(myvar_cm_average)

empty_cm_array = np.asarray(empty_cm)
empty_cm_array_1_100 = np.reshape(empty_cm_array,(1,100))
# print(cm_average_array)

df = empty_cm
df_new = pd.DataFrame(empty_cm_array_1_100,  columns=pd.MultiIndex.from_product([ df.index,df.columns]))
df_new.columns.to_flat_index()
df_new.columns   = ['_'.join(col) for col in df_new.columns.values]
print(df_new)


# Now convert combined_cms of size 30x100 to a panda dataframe

combined_cms_df = pd.DataFrame(combined_cms, columns=[df_new.columns])

   0t_0p  0t_1p  0t_2p  0t_3p  0t_4p  0t_5p  0t_6p  0t_7p  0t_8p  0t_9p  ...  \
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  ...   

   9t_0p  9t_1p  9t_2p  9t_3p  9t_4p  9t_5p  9t_6p  9t_7p  9t_8p  9t_9p  
0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  

[1 rows x 100 columns]


In [14]:
csv_filename = file_name[:-4] + ".csv"

combined_cms_df.to_csv(csv_filename)


from google.colab import files
files.download(csv_filename )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Extraneous

To reference later: 

https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb#scrollTo=UJ589fn8ST3x

To train a model with class weights:

```
class_weight = {0: weight_for_0, 1: weight_for_1}

weighted_model = make_model()
weighted_model.load_weights(initial_weights)

weighted_history = weighted_model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(val_features, val_labels),

    # The class weights go here
    class_weight=class_weight)
```

